### These Java programs simulate the running of a Pub using threads and monitors

Explanation of problem: All villagers enter a pub at some point in the day and stay for some time. The pub manager closes the pub at a specific time. Not all villagers can fit in the pub, so if they arrive and there is space, they can enter, if it is full they have to wait until they can enter or the pub closes. Villagers can finish their meal if they are in the pub when it closes.

🙂 - entered pub
😋 - ate and ready to leave
🙁 - turned down
🔒 - pub is closed

In [19]:
%%writefile village.java
class Pub {
    // YOUR CODE HERE
    int capacity, current_capacity = 0;
    boolean closed = false;
    
    // Class Invariant: (closed ∨ ¬closed) ∧ 0 ≤ current_capacity ≤ capacity
    Pub(int capacity) {
        // Precondition: capacity > 0
        this.capacity = capacity;
    }
    
    // Precondition: 0 ≤ current_capacity ≤ capacity ∧ ¬closed
    synchronized boolean enter () {
        while (current_capacity >= capacity && !closed) {
            try {wait();} catch (InterruptedException e) {}
        }
        if (closed) {return false;}
        current_capacity++;
        return true;
    }
    
    // Precondition: current_capacity > 0
    synchronized void exit () {
        if (current_capacity > 0) {
            current_capacity--;
            notify();
        }
    }
    
    // Precondition: closed = false
    synchronized void closing () {
        closed = true;
        notifyAll();
    }
}
class Villager extends Thread {
    Pub p;
    Villager(Pub p) {
        this.p = p;
    }
    public void run() {
        try {Thread.sleep((long)(Math.random() * 6000));
        } catch (Exception e) {}
        if (p.enter()) {
            System.out.print("🙂"); // entered pub
            try {Thread.sleep((long)(Math.random() * 1000)); // eating
            } catch (Exception e) {}
            System.out.print("😋"); // full and happy
            p.exit();
        } else System.out.print("🙁"); // turned down
    }
}
class Manager extends Thread {
    Pub p;
    Manager(Pub p) {
        this.p = p;
    }
    public void run() {
        try {Thread.sleep(4000);
        } catch (Exception e) {}
        System.out.print("🔒");
        p.closing();
    }
}
class Village {
    public static void main(String[] args) {
        Pub p = new Pub(8); // capacity 8
        new Manager(p).start();
        for (int i = 0; i < 20; i++) new Villager(p).start();
    }
}

Overwriting village.java


In [4]:
!javac village.java

In [5]:
!java Village

🙂🙂😋😋🙂😋🙂🙂🙂😋😋🙂🙂🙂🙂😋😋🙂🙂😋🙂🔒😋😋😋😋🙁😋🙁🙁🙁🙁🙁🙁

Explanation of problem: Same as above but when pub closes, all patrons must leave even if they're still eating.

In [32]:
%%writefile village.java
class Pub {
    // YOUR CODE HERE
    int capacity, current_capacity = 0;
    boolean closed = false;
    
    // Class Invariant: (closed ∨ ¬closed) ∧ 0 ≤ current_capacity ≤ capacity
    Pub(int capacity) {
        // Precondition: capacity > 0
        this.capacity = capacity;
    }
    
    // Precondition: 0 ≤ current_capacity ≤ capacity ∧ ¬closed
    synchronized boolean enter (Villager villager) {
        while (current_capacity >= capacity && !closed) {
            try {wait();} catch (InterruptedException e) {}
        }
        if (closed) {
            villager.interrupt();
            return false;
        }
        current_capacity++;
        return true;
    }
    
    // Precondition: current_capacity > 0
    synchronized void exit () {
        if (current_capacity > 0) {
            current_capacity--;
            notify();
        }
    }
    
    // Precondition: closed = false
    synchronized void closing () {
        closed = true;
        notifyAll();
    }
}
class Villager extends Thread {
    Pub p;
    Villager(Pub p) {
        this.p = p;
    }
    public void run() {
        try {Thread.sleep((long)(Math.random() * 6000));
        } catch (Exception e) {}
        if (p.enter(this)) {
            System.out.print("🙂"); // entered pub
            try {Thread.sleep((long)(Math.random() * 1000));
            } catch (Exception e) {System.out.print("😞"); return;}
            System.out.print("😋"); // full and happy
            p.exit();
        } else System.out.print("🙁"); // turned down
    }
}
class Manager extends Thread {
    Pub p;
    Manager(Pub p) {
        this.p = p;
    }
    public void run() {
        try {Thread.sleep(4000);
        } catch (Exception e) {}
        System.out.print("🔒");
        p.closing();
    }
}
class Village {
    public static void main(String[] args) {
        Pub p = new Pub(8); // capacity 8
        new Manager(p).start();
        for (int i = 0; i < 20; i++) new Villager(p).start();
    }
}

Overwriting village.java


In [33]:
!javac village.java

In [34]:
!java Village

🙂🙂😋😋🙂😋🙂🙂😋🙂😋🙂🙂🙂😋😋😋🙂🙂🙂😋😋😋🙂😋🙂🔒😋😋🙁🙁🙁🙁🙁🙁